In [23]:
import datasets
import numpy as np
import evaluate
import torch
import torch.nn as nn
import torch.optim as optim
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [16]:
# 下載 nltk 資源（只需一次）
nltk.download('punkt')
nltk.download("punkt_tab")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/wayne777aa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/wayne777aa/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wayne777aa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
#Load the dataset
dataset = datasets.load_from_disk("super-emotion")
train_dataset = dataset["train"]

texts = train_dataset["text"]
labels = train_dataset["labels_str"]

### Data Preprocessing

In [25]:
# NLTK prerocessing
stop_words = set(stopwords.words("english"))
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha()]  # 移除標點、數字
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

texts_cleaned = [preprocess_text(t) for t in texts[:50000]]
labels_subset = labels[:50000]

In [ ]:
# TF-IDF 向量化
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(texts_cleaned).toarray().astype(np.float32)

In [ ]:
# Label multi-hot encoding
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(labels_subset)   # shape = (n_samples, n_emotions)

# 分割訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

              precision    recall  f1-score   support

       Anger       0.90      0.72      0.80     11991
        Fear       0.88      0.67      0.76      9821
         Joy       0.91      0.82      0.86     27407
        Love       0.79      0.54      0.64      9132
     Neutral       0.61      0.18      0.28      5100
     Sadness       0.92      0.86      0.89     22308
    Surprise       0.76      0.41      0.53      3885

   micro avg       0.89      0.71      0.79     89644
   macro avg       0.82      0.60      0.68     89644
weighted avg       0.87      0.71      0.78     89644
 samples avg       0.71      0.73      0.72     89644



/home/wayne777aa/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
